# import libraries

In [1]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np


c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Read Labeled Data

In [2]:
data = pd.read_csv('../result/clustered_data.csv')

In [3]:
data['content_no_rare_words'] = data['content_no_rare_words'].apply(lambda x: ' '.join(eval(x)))

In [4]:
X = data["content_no_rare_words"]  
y = data["final_label"]
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
label_mapping

{'Acil': 0, 'Bilgilendirme': 1, 'Destek': 2, 'Çok Acil': 3}

In [5]:
data['final_label'].value_counts()

final_label
Bilgilendirme    2814
Destek           1333
Acil              605
Çok Acil          248
Name: count, dtype: int64

# Train Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train,y_train

(637                        deprem mini etek giy söyle çık
 1636              kişi hayat kaybet yar varallahim yar et
 591                            geçmiş türkiye um kayb yok
 1684    deprem artık bit san bura gizem de çatla ses p...
 2318    iskenderun devlet hastane enkaz çalış başla he...
                               ...                        
 1782    hatay afetzede gönder tır ön kes yağmala kalk ...
 1748    deprem insan enkaz çıkar halat bul yıl halat i...
 1471    asıl yıkı köylerdedirallah yardımcı olsunmedet...
 2793               marka bölge destek bekl gün birlik gün
 3858    gaziantep kahramanmaraş şanlıurfa başta doğu a...
 Name: content_no_rare_words, Length: 4000, dtype: object,
 array([1, 0, 1, ..., 1, 1, 1]))

In [7]:
print("Train sınıf oranları:", np.bincount(y_train) / len(y_train))
print("Test sınıf oranları:", np.bincount(y_test) / len(y_test))

Train sınıf oranları: [0.121   0.56275 0.26675 0.0495 ]
Test sınıf oranları: [0.121 0.563 0.266 0.05 ]


In [8]:
counts = np.bincount(y_train)
for i, count in enumerate(counts):
    print(f"{i} değeri: {count} kez")

0 değeri: 484 kez
1 değeri: 2251 kez
2 değeri: 1067 kez
3 değeri: 198 kez


In [9]:
counts = np.bincount(y_test)
for i, count in enumerate(counts):
    print(f"{i} değeri: {count} kez")

0 değeri: 121 kez
1 değeri: 563 kez
2 değeri: 266 kez
3 değeri: 50 kez


# Bert Text Classification

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')
model = BertForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-cased', num_labels=len(set(y)))
def tokenize_data(texts, tokenizer, max_length=128):
    return tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

train_encodings = tokenize_data(X_train.tolist(), tokenizer)
test_encodings = tokenize_data(X_test.tolist(), tokenizer)

c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Create Transformer Format Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, y_train.tolist())
test_dataset = CustomDataset(test_encodings, y_test.tolist())

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

optimizer = AdamW(model.parameters(), lr=2e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Train Data

In [ ]:
epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} completed with loss: {loss.item()}")

C:\Users\Enes\AppData\Local\Temp\ipykernel_24552\3472875526.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch 1 completed with loss: 0.5987384915351868
Epoch 2 completed with loss: 0.08240436017513275
Epoch 3 completed with loss: 0.17810165882110596


### Test Data Accuracy

In [ ]:
model.eval()
correct_preds = 0
total_preds = 0

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        correct_preds += (predictions == batch['labels']).sum().item()
        total_preds += batch['labels'].size(0)

accuracy = correct_preds / total_preds
print(f"Test Accuracy: {accuracy:.4f}")

C:\Users\Enes\AppData\Local\Temp\ipykernel_24552\3472875526.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Test Accuracy: 0.8370


### Predict Label

In [ ]:
def predict_label(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    predicted_label = torch.argmax(logits, dim=1).item()

    return predicted_label

input_text = "göçük altında adres adıyaman yardım"
predicted_class = predict_label(input_text)

predicted_label = {v: k for k, v in label_mapping.items()}[predicted_class]
print(predicted_label)

Destek


# Save Model

In [ ]:
torch.save(model.state_dict(), "../results/bert_model.pth")
torch.save(optimizer.state_dict(), "../results/optimizer.pth")

# All Data Test

In [ ]:
import os
folder_path = "../ParcaliSonuc"

csv_files = [file for file in os.listdir(folder_path) if file.startswith('df') and file.endswith('coordinatesspecialaddress.csv')]

dataframes = []
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df['content_no_rare_words'] = combined_df['content_no_rare_words'].apply(lambda x: ' '.join(eval(x)))

In [ ]:
combined_df['predicted_label'] = combined_df['content_no_rare_words'].apply(predict_label)

### Save All Data Result

In [ ]:
combined_df.to_csv("../results/combined_df_label_predected.csv", index=False)

# Examples

In [10]:
model_list = [
    {"name": "berturk", "tokenizer_class": BertTokenizer, "model_class": BertForSequenceClassification, "pretrained_model": "dbmdz/bert-base-turkish-cased"},
    {"name": "roberta", "tokenizer_class": RobertaTokenizer, "model_class": RobertaForSequenceClassification, "pretrained_model": "xlm-roberta-base"},
    {"name": "electra", "tokenizer_class": ElectraTokenizer, "model_class": ElectraForSequenceClassification, "pretrained_model": "dbmdz/electra-small-turkish-cased-discriminator"}
]

In [11]:
def tokenize_data(texts, tokenizer, max_length=128):
    return tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [12]:
def train_and_evaluate(model_name, tokenizer_class, model_class, pretrained_model, X_train, y_train, X_test, y_test, num_labels=4, epochs=3, batch_size=16):
    print(f"\n Start Train: {model_name}")
    tokenizer = tokenizer_class.from_pretrained(pretrained_model)
    model = model_class.from_pretrained(pretrained_model, num_labels=num_labels)

    train_encodings = tokenize_data(X_train.tolist(), tokenizer)
    test_encodings = tokenize_data(X_test.tolist(), tokenizer)

    train_dataset = CustomDataset(train_encodings, y_train.tolist())
    test_dataset = CustomDataset(test_encodings, y_test.tolist())

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    optimizer = AdamW(model.parameters(), lr=5e-5)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        model.train()
        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            optimizer.zero_grad()
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch + 1} completed with loss: {loss.item()}")

    model.eval()
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct_preds += (predictions == batch['labels']).sum().item()
            total_preds += batch['labels'].size(0)

    accuracy = correct_preds / total_preds
    print(f"{model_name} Test Accuracy: {accuracy:.4f}")
    return accuracy

In [34]:
results = {}
for model_info in model_list:
    accuracy = train_and_evaluate(
        model_name=model_info["name"],
        tokenizer_class=model_info["tokenizer_class"],
        model_class=model_info["model_class"],
        pretrained_model=model_info["pretrained_model"],
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test
    )
    results[model_info["name"]] = accuracy


 Start Train: berturk


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Enes\AppData\Local\Temp\ipykernel_15232\2219379474.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1 completed with loss: 1.0805914402008057
Epoch 2 completed with loss: 0.5718693733215332
Epoch 3 completed with loss: 0.5002769827842712
berturk Test Accuracy: 0.6936

 Start Train: bert-base


OSError: dbert-base-uncased is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
best_model = max(results, key=results.get)
print(f"\nEn iyi model: {best_model} - Accuracy: {results[best_model]:.4f}")

# LLAMA 3

In [28]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("CerebrumTech/cere-llama-3-8b-tr")
model = AutoModelForCausalLM.from_pretrained("CerebrumTech/cere-llama-3-8b-tr")

c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Enes\.cache\huggingface\hub\models--CerebrumTech--cere-llama-3-8b-tr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "CerebrumTech/cere-llama-3.1-8B-tr"

In [17]:
accuracy = train_and_evaluate("llma", AutoTokenizer, AutoModelForSequenceClassification, model_name, X_train, y_train, X_test, y_test)


 Start Train: llma


Trying to resume download...
c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Enes\.cache\huggingface\hub\models--CerebrumTech--cere-llama-3.1-8B-tr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Trying to resume download...


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'cdn-lfs-us-1.hf.co\', port=443): Max retries exceeded with url: /repos/ca/2f/ca2f6abc702e02c8aba59a6769f4fa2d96dd3b44328d06f84c69f6c760391c4b/9f728d09dd1c81dec3a98a6a781f236152d09841a239e02a2f3b23a0af0015c2?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00007.safetensors%3B+filename%3D%22model-00002-of-00007.safetensors%22%3B&Expires=1732626489&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjYyNjQ4OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2NhLzJmL2NhMmY2YWJjNzAyZTAyYzhhYmE1OWE2NzY5ZjRmYTJkOTZkZDNiNDQzMjhkMDZmODRjNjlmNmM3NjAzOTFjNGIvOWY3MjhkMDlkZDFjODFkZWMzYTk4YTZhNzgxZjIzNjE1MmQwOTg0MWEyMzllMDJhMmYzYjIzYTBhZjAwMTVjMj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=vYqWHTNy8mmPxlWsn1bIogIPt7A2g--xw8QnRob32LixfEb8-q5aACzgsmtXq55vu-a-lQeM9DeeXezmbmuBJRYtteOnZZ-EQXJDJkE9mXuzaupFi3KKudMDlR0hFqS4TgMFZwSy6sdtk0Ki9dXCtNYbXHAUDYm1j

In [ ]:
accuracy

In [22]:
predicted_labels = []
for text in X_test:  
    predicted_class = predict_label(text) 
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    predicted_labels.append(predicted_label)

for input_text, predicted_label in zip(X_test, predicted_labels):
    print(f"Girdi: {input_text}\nTahmin Edilen Etiket: {predicted_label}\n")

Girdi: ümit doğanbaşımız sağ
Tahmin Edilen Etiket: Bilgilendirme

Girdi: hatay antakya cebrail mahalle daloğlu apartman yad civar tanı öğretmen baba ulaş konu yardımcı ol
Tahmin Edilen Etiket: Destek

Girdi: destek ol
Tahmin Edilen Etiket: Bilgilendirme

Girdi: paylaş acil
Tahmin Edilen Etiket: Çok Acil

Girdi: kar kış göçük alt kal kal cümle insan yar eyle yarabbimgeçmiş türkiye
Tahmin Edilen Etiket: Acil

Girdi: hatay durum kötü yar gi̇tmı̇yor enkaz alt kalıp şua öl hi̇potermi̇den öl
Tahmin Edilen Etiket: Acil

Girdi: elazığ akşam elazığ pavyon bi̇rahaneleri̇n sokak özellikle gez kapalı canlı müzik yap cafe kapalı alkol satış yap büfe bugün aç güzel mi̇lleti̇z ac bi̇rli̇kte yaş şan yüce rabbi bi̇rli̇gi̇mi̇zı̇ artır
Tahmin Edilen Etiket: Bilgilendirme

Girdi: tit alt ikinci üçüncü sonra yorum yar feryat adresle yakar delir kısıtla twitter
Tahmin Edilen Etiket: Destek

Girdi: deprem bölge insan psikolojik destek ver
Tahmin Edilen Etiket: Bilgilendirme

Girdi: fransa gönder yar ekib böl